### Import Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import numpy as np

###  dataset path

In [6]:
root_dir = "C:/Users/khade/Downloads/leavesphotos"  # Change this to your dataset folder

# Load dataset (Train & Validation)
batch_size = 32
img_size = (224, 224)

train_dataset = image_dataset_from_directory(
    root_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_dataset = image_dataset_from_directory(
    root_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)


Found 6900 files belonging to 80 classes.
Using 5520 files for training.
Found 6900 files belonging to 80 classes.
Using 1380 files for validation.


### Class names

In [8]:
class_names = train_dataset.class_names
num_classes = len(class_names)
print(f"Classes: {class_names}, Total Classes: {num_classes}")

# Normalize and preprocess images
def preprocess(image, label):
    image = preprocess_input(image)  # Apply EfficientNet preprocessing
    return image, label

train_dataset = train_dataset.map(preprocess).cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess).cache().prefetch(buffer_size=tf.data.AUTOTUNE)


Classes: ['Aloevera', 'Amla', 'Amruthaballi', 'Arali', 'Astma_weed', 'Badipala', 'Balloon_Vine', 'Bamboo', 'Beans', 'Betel', 'Bhrami', 'Bringaraja', 'Caricature', 'Castor', 'Catharanthus', 'Chakte', 'Chilly', 'Citron lime (herelikai)', 'Coffee', 'Common rue(naagdalli)', 'Coriender', 'Curry', 'Doddpathre', 'Drumstick', 'Ekka', 'Eucalyptus', 'Ganigale', 'Ganike', 'Gasagase', 'Ginger', 'Globe Amarnath', 'Guava', 'Henna', 'Hibiscus', 'Honge', 'Insulin', 'Jackfruit', 'Jasmine', 'Kambajala', 'Kasambruga', 'Kohlrabi', 'Lantana', 'Lemon', 'Lemongrass', 'Malabar_Nut', 'Malabar_Spinach', 'Mango', 'Marigold', 'Mint', 'Neem', 'Nelavembu', 'Nerale', 'Nooni', 'Onion', 'Padri', 'Palak(Spinach)', 'Papaya', 'Parijatha', 'Pea', 'Pepper', 'Pomoegranate', 'Pumpkin', 'Raddish', 'Rose', 'Sampige', 'Sapota', 'Seethaashoka', 'Seethapala', 'Spinach1', 'Tamarind', 'Taro', 'Tecoma', 'Thumbe', 'Tomato', 'Tulsi', 'Turmeric', 'ashoka', 'camphor', 'kamakasturi', 'kepala'], Total Classes: 80


### Load pre-trained EfficientNetB0

In [11]:
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(num_classes, activation="softmax")(x)  # Multi-class classification

# Create final model
model = Model(inputs=base_model.input, outputs=x)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling (Rescaling)         │ (None, 224, 224, 3)       │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization (Normalization) │ (None, 224, 224, 3)       │               7 │ rescaling[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_1 (Rescaling)       │ (None, 224, 224, 3)       │               0 │ normalization[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 225, 225, 3)       │               0 │ rescaling_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 112, 112, 32)      │             864 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 112, 112, 32)      │             128 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 112, 112, 32)      │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 112, 112, 32)      │             288 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 112, 112, 32)      │             128 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 112, 112, 32)      │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 32)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 32)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 8)           │             264 │ block1a_se_reshape[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_expand (Conv2D)    │ (None, 1, 1, 32)          │             288 │ block1a_se_reduce[0][0]    │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 4,223,859 (16.11 MB)

 Trainable params: 174,288 (680.81 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

### Train the model

In [13]:
epochs = 3  # Adjust as needed
history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)


Epoch 1/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 220s 1s/step - accuracy: 0.2622 - loss: 3.3740 - val_accuracy: 0.7514 - val_loss: 1.1355
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 182s 1s/step - accuracy: 0.8182 - loss: 0.8537 - val_accuracy: 0.8391 - val_loss: 0.6439
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 182s 1s/step - accuracy: 0.9180 - loss: 0.4163 - val_accuracy: 0.8877 - val_loss: 0.4598


### Load test dataset 

In [ ]:
test_dataset = image_dataset_from_directory(
    root_dir,
    image_size=img_size,
    batch_size=batch_size
)

# Preprocess test dataset
test_dataset = test_dataset.map(preprocess).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Evaluate model
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")


Found 6900 files belonging to 80 classes.
118/216 ━━━━━━━━━━━━━━━━━━━━ 1:27 896ms/step - accuracy: 0.9521 - loss: 0.2682

### Get true labels and predictions and classification report

In [5]:
y_true, y_pred = [], []

for images, labels in test_dataset:
    preds = model.predict(images)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(labels.numpy())
    
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))


1/1 [==============================] - 3s 3s/step
               precision    recall  f1-score   support

     Aloevera     0.9760    0.9939    0.9849       164
         Amla     0.9667    0.9932    0.9797       146
 Amruta_Balli     0.9424    0.8973    0.9193       146
        Arali     0.9932    0.9932    0.9932       146
       Ashoka     1.0000    0.9863    0.9931       146
  Ashwagandha     0.9732    0.9932    0.9831       146
      Avacado     0.9728    0.9795    0.9761       146
       Bamboo     0.9932    1.0000    0.9966       146
       Basale     0.9333    0.9589    0.9459       146
        Betel     0.9861    0.9404    0.9627       151
    Betel_Nut     1.0000    1.0000    1.0000       146
       Brahmi     0.9730    0.9863    0.9796       146
       Castor     0.9876    0.9938    0.9907       160
   Curry_Leaf     0.9928    0.9452    0.9684       146
   Doddapatre     0.9799    1.0000    0.9898       146
         Ekka     0.9660    0.9726    0.9693       146
       Ganike 

### Model saveing

In [7]:
model.save("C:/Users/khade/OneDrive/Desktop/Medicinal_Plant/UI/models/EfficientNetModel.h5")
print("Model saved as 'EfficientNetModel.h5'")


C:\Users\dell\anaconda3\envs\multi\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as 'EfficientNetModel.h5'
